## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-21-22-50-09 +0000,nyt,"In a Reversal, Trump Will Not Meet With Putin ...",https://www.nytimes.com/2025/10/21/us/politics...
1,2025-10-21-22-49-29 +0000,nypost,Andrew Cuomo torches Zohran Mamdani over lack ...,https://nypost.com/2025/10/21/us-news/andrew-c...
2,2025-10-21-22-47-00 +0000,wsj,BlackRock Among Biggest Investors in Meta’s Gi...,https://www.wsj.com/finance/investing/blackroc...
3,2025-10-21-22-43-27 +0000,nypost,Theme park superfan Travis Kelce joins activis...,https://nypost.com/2025/10/21/business/travis-...
4,2025-10-21-22-39-34 +0000,nyt,‘KPop Demon Hunters’ Helps Lift Netflix Revenu...,https://www.nytimes.com/2025/10/21/business/me...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2348/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
1,trump,56
106,house,17
132,up,16
105,white,14
158,hamas,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
135,2025-10-21-17-27-00 +0000,wsj,Key Senate Republicans are calling for the wit...,https://www.wsj.com/politics/policy/paul-ingra...,139
226,2025-10-21-11-43-48 +0000,cbc,White House demolition work begins as Trump pu...,https://www.cbc.ca/news/world/us-white-house-d...,111
115,2025-10-21-18-44-04 +0000,bbc,From bowling alley to ballroom - Trump isn't t...,https://www.bbc.com/news/articles/cr7m2v80dy4o...,110
127,2025-10-21-17-55-57 +0000,nypost,Hillary mocked after blasting Trump’s White Ho...,https://nypost.com/2025/10/21/us-news/hillary-...,108
311,2025-10-21-02-48-43 +0000,bbc,White House begins demolishing part of East Wi...,https://www.bbc.com/news/articles/ced6np51532o...,108


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
135,139,2025-10-21-17-27-00 +0000,wsj,Key Senate Republicans are calling for the wit...,https://www.wsj.com/politics/policy/paul-ingra...
15,72,2025-10-21-22-12-51 +0000,nyt,Vance Lands in Israel as U.S. Tries to Shore U...,https://www.nytimes.com/2025/10/21/world/middl...
344,55,2025-10-20-23-00-00 +0000,wsj,Business and political leaders have intensifie...,https://www.wsj.com/politics/elections/nyc-may...
306,55,2025-10-21-04-53-16 +0000,wapo,"Japan elects first female leader, a hawk who p...",https://www.washingtonpost.com/world/2025/10/2...
149,40,2025-10-21-17-02-40 +0000,nypost,Top Trump adviser boldly predicts Democrats wi...,https://nypost.com/2025/10/21/us-news/top-trum...
207,40,2025-10-21-13-25-26 +0000,nypost,Former French President Sarkozy begins 5-year ...,https://nypost.com/2025/10/21/world-news/forme...
17,40,2025-10-21-22-11-36 +0000,nyt,Paris Tour Guide Saw Stolen Louvre Jewels Just...,https://www.nytimes.com/2025/10/21/style/louvr...
128,39,2025-10-21-17-44-24 +0000,nypost,Hamas released bodies of two more dead hostage...,https://nypost.com/2025/10/21/world-news/hamas...
202,39,2025-10-21-13-56-46 +0000,nypost,Warner Bros. Discovery says it’s open to a sal...,https://nypost.com/2025/10/21/business/warner-...
46,34,2025-10-21-21-15-07 +0000,nypost,California dad who drove family car across flo...,https://nypost.com/2025/10/21/us-news/barstow-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
